### Lesson 2. Keras

1. Обучить, нейронную сеть на Keras на датасете MNIST.

In [1]:
# pip install keras
# pip install mnist
#!pip install scikeras[tensorflow]

In [51]:
import numpy as np
import pandas as pd
import mnist
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

### Building model for digit recognition

In [18]:
#import images
train_images = mnist.train_images()
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()

# Normalize the images.
train_images = (train_images / 255)
test_images = (test_images / 255)

# Flatten the images.
train_images = train_images.reshape((-1, 784))
test_images = test_images.reshape((-1, 784))

In [32]:
# Build the model.
def create_model(optimizer='adam'):
    model = Sequential([
      Dense(64, input_shape=(784,), activation='relu'),
      Dense(32, activation='relu'),
      Dense(10, activation='softmax')
    ])

    # Compile the model.
    model.compile(
      optimizer=optimizer,
      loss='categorical_crossentropy',
      metrics=['accuracy'],
    )
    return model

In [38]:
model = create_model()

# Train the model.
model.fit(train_images, to_categorical(train_labels), epochs=5, batch_size=32)

# Evaluate the model.
loss,metrics = model.evaluate(test_images, to_categorical(test_labels))
print('*****************************')
print(f'Loss - {np.round(loss,2)}, Metrics - {np.round(metrics,2)}')

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2984 - accuracy: 0.9150
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1355 - accuracy: 0.9596
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0962 - accuracy: 0.9705
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0784 - accuracy: 0.9764
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0979 - accuracy: 0.9715
*****************************
Loss - 0.1, Metrics - 0.97


In [39]:
# Predict on the first 10 test images.
predictions = model.predict(test_images[:10])

# Print our model's predictions.
print(f'Predictions: {np.argmax(predictions, axis=1)}')

# Check our predictions against the ground truths.
print(f'True labels: {test_labels[:10]}')

1/1 [==============================] - 0s 53ms/step
Predictions: [7 2 1 0 4 1 4 9 5 9]
True labels: [7 2 1 0 4 1 4 9 5 9]


### Picking params via GridSearchCV

In [48]:
batch_size = [32,60,100]
epochs = [5, 10, 15]
optimizer = ['SGD', 'RMSprop', 'Adam']

In [ ]:
model = KerasClassifier(create_model)
params= dict(batch_size=batch_size, epochs=epochs, optimizer=optimizer)
grid = GridSearchCV(model, params, refit=False, cv=3, scoring='accuracy')

grid_result = grid.fit(train_images, to_categorical(train_labels))
print(grid_result.best_score_, grid_result.best_params_)

In [55]:
print(grid_result.best_score_, grid_result.best_params_)

0.9688500000000001 {'batch_size': 32, 'epochs': 15, 'optimizer': 'SGD'}


In [56]:
df = pd.DataFrame(grid_result.cv_results_)

In [69]:
df.sort_values(['mean_test_score'],ascending=False)[['param_batch_size','param_epochs','param_optimizer','mean_test_score']].head(10)

,param_batch_size,param_epochs,param_optimizer,mean_test_score
6,32,15,SGD,0.968850
5,32,10,Adam,0.968600
15,60,15,SGD,0.968133
17,60,15,Adam,0.967867
12,60,10,SGD,0.967317
26,100,15,Adam,0.967233
21,100,10,SGD,0.967200
25,100,15,RMSprop,0.967067
16,60,15,RMSprop,0.967033
7,32,15,RMSprop,0.966717
